The main reason for me to taking up this project/mission was my obvious interest and love for stand up comedy and also because i wanted to know why exactly do i love hasan minhaj's content. What is it that he does so differently from others? 

Anyway, we are about to find that out! As i was going the process of building this project and making a workflow, i realised how complex this could turn out to be. My main questions i wanted to be answered were...
1. How the comedians differ from each other? [This is observed from the fulldata dataframe's analysis, as it contains transcripts of 16 comedians] 
2. How each comedians' content differ between their own stand up specials [This analysis will be addressed by amy schumer, john mulaney and kevin hart's transcripts from various specials]
3. What is the style of hasan minhaj and how does he express it? [From the data scraped from his 22 episodes from his Netflix Special called Patriot Act]

I will have to release the analysis in phases as it is massive and in the first phase, i am putting out the data extraction and intense cleaning process in two different notebooks for ease of use. 

### Data Extraction

The data has been scraped from two websites 
1. Scraps From The Loft (http://scrapsfromtheloft.com) 
2. yousubtitles (https://www.yousubtitles.com/Patriot-Act-cd-149803/) 
The second website provides subtitles, transcripts for videos on youtube and thankfully hasan releases his episodes on youtube in addition to Netflix which enabled me to get the transcripts manually and store them in a folder called hasantrans 

In [1]:
import re
import string
import requests
from bs4 import BeautifulSoup
import pickle
import os
import pandas as pd
pd.set_option('max_colwidth',150)

In [2]:
# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="post-content").find_all('p')]
    return text

# URLs of transcripts in scope
urls = ['https://scrapsfromtheloft.com/2019/12/05/tiffany-haddish-black-mitzvah-transcript/',
        'https://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
        'https://scrapsfromtheloft.com/2018/05/05/john-mulaney-kid-gorgeous-at-radio-city-full-transcript/',
        'http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        'https://scrapsfromtheloft.com/2018/05/15/ali-wong-hard-knock-wife-full-transcript/',
       'https://scrapsfromtheloft.com/2020/03/01/pete-davidson-alive-from-new-york-transcript/',
       'https://scrapsfromtheloft.com/2019/04/05/kevin-hart-irresponsible-transcript/',
       'https://scrapsfromtheloft.com/2019/03/23/amy-schumer-growing-transcript/',
       'https://scrapsfromtheloft.com/2019/02/17/ken-jeong-you-complete-me-ho-transcript/',
       'https://scrapsfromtheloft.com/2018/12/22/ellen-degeneres-relatable-transcript/',
       'https://scrapsfromtheloft.com/2018/12/15/vir-das-losing-it-transcript/',
       'https://scrapsfromtheloft.com/2018/11/21/trevor-noah-son-of-patricia-transcript/',
       'https://scrapsfromtheloft.com/2018/10/26/adam-sandler-100-fresh-transcript/',
       'https://scrapsfromtheloft.com/2020/05/10/russell-peters-deported-transcript/',
        'https://scrapsfromtheloft.com/2020/06/01/kenny-sebastian-dont-be-that-guy-transcript/',
        'https://scrapsfromtheloft.com/2017/08/30/aziz-ansari-intimate-moments-sensual-evening-2010-full-transcript/']

# Comedian names 
comedians = ['Tiffany Haddish', 'Ricky Gervais', 'John Mulaney', 'Hasan Minhaj', 'Ali Wong', 'Pete Davidson',
          'Kevin Hart', 'Amy Schumer','Ken Jeong', 'Ellen Degeneres','Vir Das','Trevor Noah','Adam Sandler',
          'Russell Peters','Kenny Sebastian','Aziz Ansari']

In [3]:
#To study the variation in comic delivery over years

kevinurls = ['https://scrapsfromtheloft.com/2019/08/08/kevin-hart-gun-compartment-transcript/',
             'https://scrapsfromtheloft.com/2019/04/05/kevin-hart-irresponsible-transcript/',
            'https://scrapsfromtheloft.com/2018/01/14/kevin-hart-what-now-2016-full-transcript/',
            'https://scrapsfromtheloft.com/2018/01/14/kevin-hart-let-me-explain-2013-full-transcript/',
            'https://scrapsfromtheloft.com/2017/09/02/kevin-hart-seriously-funny-2010-full-transcript/' ]
kevinindex = ['2020','2019','2016','2013','2010']

amyurls = ['https://scrapsfromtheloft.com/2017/09/08/amy-schumer-mostly-sex-stuff-2012-full-transcript/',
          'https://scrapsfromtheloft.com/2017/09/05/amy-schumer-live-apollo-2015-full-transcript/',
          'https://scrapsfromtheloft.com/2017/06/20/amy-schumer-leather-special-2017-full-transcript/',
          'https://scrapsfromtheloft.com/2019/03/23/amy-schumer-growing-transcript/']
amyindex = ['2012','2015','2017','2019']

johnurls = ['https://scrapsfromtheloft.com/2018/09/23/john-mulaney-snl-monologue-2018-transcript/',
           'https://scrapsfromtheloft.com/2018/05/05/john-mulaney-kid-gorgeous-at-radio-city-full-transcript/',
           'https://scrapsfromtheloft.com/2017/09/25/john-mulaney-new-in-town-2012-full-transcript/',
           'https://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/']
johnindex = ['2019','2018','2012','2017']

In [4]:
# To analyse the variations and also the content and any other latent features and simply because i like 
# Hasan Minhaj Transcripts - Patriot Act From the URL: https://www.yousubtitles.com/Patriot-Act-cd-149803/
# The text files have been dowloaded and saved into a folder called hasantrans 

directory = 'hasantrans'
episodes = []
texts = []
for filename in os.listdir(directory):
    f = open("hasantrans/"+ filename,"r",encoding="utf8")
    lines = f.readlines()
    epname = filename.split('-Patriot')[0].replace('-',' ')
    episodes.append(epname)
    actext = ' '.join(lines)
    texts.append(actext)

#### DATA LOADING

In [5]:
# Loading everyones' transcripts into a list called transcripts and doing the same for kevin, amy and john
transcripts = [url_to_transcript(u) for u in urls]
print(len(transcripts),' transcripts extracted')
kevintranscripts = [url_to_transcript(u) for u in kevinurls]
print(len(kevintranscripts),' transcripts extracted')
amytranscripts = [url_to_transcript(u) for u in amyurls]
print(len(amytranscripts),' transcripts extracted')
johntranscripts = [url_to_transcript(u) for u in johnurls]
print(len(johntranscripts),' transcripts extracted')

16  transcripts extracted
5  transcripts extracted
4  transcripts extracted
4  transcripts extracted


In [6]:
# Making separate dataframes for the comedians as they are to serve different purposes. Hence we have 5 dataframes in the end. 
# One having all the comedians, to compare and analyse. Other 4 corresponding to kevin, amy, john and hasan to be analysed
# individually 

datadict = {}
for i, c in enumerate(comedians):
    datadict[c] = ' '.join(transcripts[i])

kevindatadict = {}
for i, c in enumerate(kevinindex):
    kevindatadict[c] = ' '.join(kevintranscripts[i])

amydatadict = {}
for i, c in enumerate(amyindex):
    amydatadict[c] = ' '.join(amytranscripts[i])

johndatadict = {}
for i, c in enumerate(johnindex):
    johndatadict[c] = ' '.join(johntranscripts[i])

hasandata = pd.DataFrame({'episode':episodes,'transcript': texts},index = episodes) #My favorite 

In [7]:
#The data is made ready for cleaning and analysis

fulldata = pd.DataFrame.from_dict(datadict,orient = 'index', columns = ['transcript'])

kevindata = pd.DataFrame.from_dict(kevindatadict,orient = 'index', columns = ['transcript']) #extreme swear words usage

amydata = pd.DataFrame.from_dict(amydatadict,orient = 'index', columns = ['transcript']) #Complete positive script 

johndata = pd.DataFrame.from_dict(johndatadict,orient = 'index', columns = ['transcript']) #moderate and balanced script 

In [8]:
# Save all the raw dataframes 

fulldata.to_pickle("rawfulldata.pkl")
kevindata.to_pickle("rawkevindata.pkl")
amydata.to_pickle("rawamydata.pkl")
johndata.to_pickle("rawjohndata.pkl")
hasandata.to_pickle("rawhasandata.pkl")